In [6]:
import utility 
import numpy as np
from importlib import reload
reload(utility)
import sys, os
import time
from tqdm import tqdm
import matplotlib.pyplot as pl

HOME = '/cluster/home/ajoudaki/projects2019-string-embedding/python_tools'

file_name = 'seqs8'
data_path = HOME + '/data/' + file_name + '.npz'
result_path = HOME + '/results/' + file_name 
index_path = result_path + '/index/'
search_path = result_path + '/search/'

if not os.path.exists(result_path):
    os.makedirs(result_path)
if not os.path.exists(index_path):
    os.makedirs(index_path)
if not os.path.exists(search_path):
    os.makedirs(search_path)


Res = np.load(data_path)


seqs = Res['seqs']
vals = Res['vals']
options = Res['options']
Op = (options[()])
if int(file_name[4:])>=7:
    gene_lens = Res['gene_lens']
    num_seqs = Res['num_seqs']
else:
    num_seqs = [Op.num_seq]*Op.repeat

print('num seqs = ', len(seqs), ' mean lenth of seqs = ', np.mean([len(seqs[i]) for i in range(len(seqs))]))
print(Op)

num seqs =  10  mean lenth of seqs =  900716.1
{'mutation_rate': 0.1, 'gene_len': 100, 'num_genes': 200, 'padding': 2000, 'num_seq': 5, 'repeat': 2, 'geometric_p': 0.4, 'reverse_p': 0, 'gene_len2': 1000, 'num_seq2': 5, 'colored': False, 'print_values': False, 'print_seqs': False, 'save_directory': '../data/seqs8'}


In [ ]:
proj_dim = 120
k_small = 4 
k_big = 100
num_trees = 2
num_neighbours = 200
step_build = 5
step_search = 5
metric = 'euclidean'


kmers, s_kmer_vals, kmer_pos, kmer_seq_id = utility.list_kmers_simple(seqs, vals = vals,  
                                         k = k_small, addy = True, padding = int(k_big))
kmer_vals = utility.get_kmver_vals(s_kmer_vals, k_big)
build_indices = np.arange(0,len(kmers),step_build)
search_indices = np.arange(0,len(kmers),step_search)

convolved = utility.random_projection(kmers, k_big, proj_dim)

knn_index = utility.build_index(matrix = convolved, 
                                indices = build_indices, 
                                num_trees = num_trees, 
                                metric = metric)

knn_index.save(index_path + '0.ann')
index = AnnoyIndex(proj_dim, metric= metric) 
index.load(index_path + '0.ann')
NN,NN_dist = utility.knn_search_value(index, convolved, search_indices, num_neighbours)
    
quad = utility.eval_results(search_indices, NN, kmers, kmer_vals, kmer_pos, kmer_seq_id, num_seqs, options)


print('hit ratio: ', len(quad)/len(search_indices)/num_neighbours)

computing the list of kmers and values 


100%|██████████| 10/10 [00:06<00:00,  1.43it/s]


 computing the projection


100%|██████████| 9009151/9009151 [00:11<00:00, 780091.95it/s]

 computing the convolution



100%|██████████| 120/120 [01:31<00:00,  1.35it/s]


time to build 2 trees = 37.35229301452637
nearest neighbours search:


 67%|██████▋   | 1214811/1801831 [09:23<02:25, 4033.17it/s]

In [ ]:
# print(Op)